In [4]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from collections import Counter
from nltk.corpus import stopwords
import math
import numpy as np
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aaditya11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def docPreProcessing(filepath):
    stopwords_dict = Counter(stopwords.words('english'))

    file = open(filepath, 'r', errors='replace')
    lines = file.readlines()


    stopwords_dict = Counter(stopwords.words('english'))
    
    
    doctext = ""
    for line in lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).strip().lower()
        line = ' '.join([word for word in line.split() if word not in stopwords_dict])
        doctext += line
        
    doctextList = doctext.split()
    return doctextList

In [9]:
def tf(wordlist):

    wordmap = {}
    
    for word in wordlist:
        if word in wordmap:
            wordmap[word] += 1
        else:
            wordmap[word] = 1
            
    
    numWords = len(wordlist)
    
    for word in wordmap:
        wordmap[word] = wordmap[word] / numWords
    
    return wordmap
        


In [25]:
import time

# //calculate average time taken for 100 runs 
total_time = 0
for i in range(100):
    start = time.process_time()
    wordmap = tf(wordlist = docPreProcessing('../testset/sci.space/60151'))   
    total_time += (time.process_time() - start)*1000000

print(total_time/100)






840.2805000000012


In [29]:
def tfidfMapBuilder(doclist):
    termset = set()

    for doc in doclist:
        termset.update(set(docPreProcessing(doc)))

    df_columns = doclist.copy()
    df_columns.append('df')

    df = pd.DataFrame(0, index=list(termset), columns=df_columns)

    for doc in doclist:
        wordmap = tf(docPreProcessing(doc))
        for term in wordmap:
            df.at[term, doc] = wordmap[term]
            df.at[term, 'df'] += 1

    N = len(doclist)

    return df

In [30]:
def mapToMatrix(df, N):
    df['df'] = df['df'].apply(lambda x: math.log(N/x))

    for index, row in df.iterrows():
        term_df = row['df']
        row = row.apply(lambda x: x * term_df)
    
    df = df.drop(columns=['df'], axis=1)

    return df

In [31]:
def cosineSimilarityScore(q, D):
    q_mag = np.sqrt(q.dot(q))
    
    qT = np.reshape(q, (1, q.shape[0]))
    qTD = np.matmul(qT, D).reshape((D.shape[1], ))
    
    D_mags = np.sqrt(np.sum(D*D, axis=0))
    divisors = q_mag * D_mags
    cos_thetas = np.divide(qTD, divisors)
    
    cos_thetas = np.clip(cos_thetas, -1, 1)
    
    scores = np.arccos(cos_thetas)
    
    return scores
    

In [50]:
def fileCollector(path):
    filelist = []
    
    for (root, dirs, files) in os.walk(path, topdown=True):
        for file in files:
            filelist.append(os.path.join(root, file))
            
            if len(filelist) == 100:
                return filelist

    return filelist

In [51]:
doclist = []

doclist.append('query.txt')

doclist.extend(fileCollector(r'../testset/sci.space'))

tfidfDataFrame = mapToMatrix(tfidfMapBuilder(doclist), len(doclist))

# tfidfDataFrame
# tfidfMapBuilder.to_csv('tfidf.csv')

doc_tfidfMatrix = tfidfDataFrame.loc[:, tfidfDataFrame.columns != 'query.txt'].to_numpy().round(decimals=4)
query_vector = tfidfDataFrame.loc[:, "query.txt"].to_numpy().round(decimals=4)

In [52]:
scorelist = cosineSimilarityScore(query_vector, doc_tfidfMatrix)
scoremap = {}

for index, score in enumerate(scorelist):
    scoremap[tfidfDataFrame.columns[index + 1]] = score

scoremap = dict(sorted(scoremap.items(), key=lambda item:item[1]))

for doc in list(scoremap.items())[:5]:
    print(doc[0] + " : " + str(doc[1]))


../testset/sci.space/60827 : 1.4725654859548984
../testset/sci.space/61138 : 1.473050747060915
../testset/sci.space/61288 : 1.512040504079174
../testset/sci.space/60168 : 1.5331981328975341
../testset/sci.space/61339 : 1.5389541866018606
